In [8]:
import pandas as pd
import numpy as np
from scipy import stats
import math
from sklearn.ensemble import RandomForestRegressor
import warnings 
warnings.filterwarnings("ignore")

In [9]:
test = pd.read_csv('TestClean.csv',index_col=0)
train = pd.read_csv('TrainClean.csv',index_col=0)

In [31]:
def predictSales(to_return,model1,model2,test,stores,months):
    for store in stores:
        temp = test.loc[test['StoreID'] == store]
        for month in months:
            X = temp.loc[temp[month] == 1]
            X = X.drop(labels='StoreID', axis=1)
            customers = model1.predict(X)
            X['NumberOfCustomers'] = customers
            res = model2.predict(X)
            res = np.expm1(res)
            res = np.round(res)
            if(month == 'March'):
                   res = {'StoreID' : [store], 'Month' : [3], 'NumberOfSales' : [np.sum(res)]}
            else:
                   res = {'StoreID' : [store], 'Month' : [4], 'NumberOfSales' : [np.sum(res)]}
            res = pd.DataFrame(res)
            to_return = to_return.append(res,ignore_index=True)
    
    to_return.to_csv("Results.csv")

In [12]:
variablesC = train.columns.drop('NumberOfCustomers')
variablesC = variablesC.drop('NumberOfSales')
variablesS =  train.columns.drop('NumberOfSales')
        
var = train[variablesC]
tar = train['NumberOfCustomers']
    
model1 = RandomForestRegressor(random_state=12345,min_samples_leaf=5) #model 1 customers
model1.set_params(n_estimators=150)
model1.fit(var, tar)
    
var = train[variablesS]
tar = train['NumberOfSales']
    
model2 = RandomForestRegressor(random_state=12345,min_samples_leaf=5) #model 2 sales
model2.set_params(n_estimators=150)
model2.fit(var, tar)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
           oob_score=False, random_state=12345, verbose=0,
           warm_start=False)

In [33]:
stores = test['StoreID'].unique()
months = ['March','April']
df = pd.DataFrame(columns=['StoreID', 'Month', 'NumberOfSales'])
predictSales(df,model1,model2,test,stores,months)